In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import json
from utils.experiment import Experiment

In [2]:
dataset = 'point_mnist'
slice_setting = 'over'

In [3]:
if dataset == 'point_mnist':
    df_train = pd.read_csv('../dataset/pointcloud_mnist_2d/train.csv')

    X = df_train[df_train.columns[1:]].to_numpy()
    y = df_train[df_train.columns[0]].to_numpy()

    X = X.reshape(X.shape[0], -1, 3)
    
    num_points = np.sum((X[:, :, 2] > 0).astype(int), axis=1)
    
    set_size_median = np.median(num_points).astype(int)
    n_slices = 8 if slice_setting == 'over' else 2

elif dataset == 'modelnet40':
    set_size_median = 512
    n_slices = 16 if slice_setting == 'over' else 3
    
elif dataset == 'oxford':
    with open('../dataset/oxford/train_test_AE8.pkl', 'rb') as f:
        data = pickle.load(f)

    X_train, y_train, X_test, y_test, classnames = data

    num_points = np.array([i.shape[0] for i in X_train])

    set_size_median = np.median(num_points).astype(int)
    n_slices = 128 if slice_setting == 'over' else 8

print(dataset, set_size_median, n_slices)

point_mnist 150 8


In [4]:
code_length = 1024
ref = 'rand'
seeds = [0, 1, 4, 10, 16]
ks = [4, 8, 16]
reports = []

### FS

In [5]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'fs', 'faiss-lsh', 
                         random_state=seed, ref_func=ref, k=k, ref_size=set_size_median, code_length=code_length)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 3645.01it/s]


{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.0002970361948013306, 'inf_time_per_sample': 0.00038264269828796384, 'acc': 0.8021, 'precision_k': 0.7509}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4023.79it/s]


{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.00026838419437408447, 'inf_time_per_sample': 0.00045978360176086424, 'acc': 0.8107, 'precision_k': 0.739475}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.0002567981004714966, 'inf_time_per_sample': 0.0003881385326385498, 'acc': 0.794, 'precision_k': 0.74525}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4156.35it/s]


{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.0002594175338745117, 'inf_time_per_sample': 0.0003754882097244263, 'acc': 0.8041, 'precision_k': 0.7334}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4133.78it/s]


{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'emb_time_per_sample': 0.00027086079120635986, 'inf_time_per_sample': 0.0003759662866592407, 'acc': 0.806, 'precision_k': 0.7188}


### SWE

In [6]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'swe', 'faiss-lsh', random_state=seed, ref_func=ref, k=k, ref_size=set_size_median, code_length=code_length, num_slices=n_slices)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2177.16it/s]


{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.0004803830862045288, 'inf_time_per_sample': 0.0003893955945968628, 'acc': 0.9197, 'precision_k': 0.89705}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2105.41it/s]


{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.0004948450088500977, 'inf_time_per_sample': 0.00038885202407836915, 'acc': 0.9202, 'precision_k': 0.8836125}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.00048592169284820556, 'inf_time_per_sample': 0.00038374199867248536, 'acc': 0.9086, 'precision_k': 0.87685}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2007.43it/s]


{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.0005215465068817139, 'inf_time_per_sample': 0.00039430990219116213, 'acc': 0.9057, 'precision_k': 0.861775}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1867.03it/s]


{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'emb_time_per_sample': 0.0005565723180770874, 'inf_time_per_sample': 0.0003972540855407715, 'acc': 0.9003, 'precision_k': 0.8424125}


### WE

In [7]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'we', 'faiss-lsh', 
                         random_state=seed, ref_func=ref, k=k, ref_size=set_size_median, code_length=code_length)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:25<00:00, 397.61it/s]


{'dataset': 'point_mnist', 'pooling': 'we', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.002533702611923218, 'inf_time_per_sample': 0.0003858808994293213, 'acc': 0.9209, 'precision_k': 0.89005}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:24<00:00, 416.57it/s]


{'dataset': 'point_mnist', 'pooling': 'we', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.002421518397331238, 'inf_time_per_sample': 0.0003738917827606201, 'acc': 0.9224, 'precision_k': 0.87765}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'we', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 0.0023444679021835326, 'inf_time_per_sample': 0.0003768328905105591, 'acc': 0.9203, 'precision_k': 0.887925}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:23<00:00, 419.16it/s]


{'dataset': 'point_mnist', 'pooling': 'we', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 0.002405758810043335, 'inf_time_per_sample': 0.00038958652019500733, 'acc': 0.9193, 'precision_k': 0.8766375}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:26<00:00, 384.23it/s]


{'dataset': 'point_mnist', 'pooling': 'we', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'emb_time_per_sample': 0.00262330219745636, 'inf_time_per_sample': 0.00038140270709991457, 'acc': 0.915, 'precision_k': 0.862525}


### Cov

In [8]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'cov', 'faiss-lsh',
                         random_state=seed, k=k, ref_size=set_size_median, code_length=code_length)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 19674.05it/s]


{'dataset': 'point_mnist', 'pooling': 'cov', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 7.202529907226563e-05, 'inf_time_per_sample': 0.00038690080642700197, 'acc': 0.2648, 'precision_k': 0.2473}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17041.20it/s]


{'dataset': 'point_mnist', 'pooling': 'cov', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 7.7789306640625e-05, 'inf_time_per_sample': 0.0003719609260559082, 'acc': 0.276, 'precision_k': 0.248125}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'cov', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'emb_time_per_sample': 7.228522300720214e-05, 'inf_time_per_sample': 0.00038124232292175293, 'acc': 0.2648, 'precision_k': 0.2473}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17056.26it/s]


{'dataset': 'point_mnist', 'pooling': 'cov', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'emb_time_per_sample': 7.982578277587891e-05, 'inf_time_per_sample': 0.0003696885824203491, 'acc': 0.276, 'precision_k': 0.248125}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 20119.46it/s]


{'dataset': 'point_mnist', 'pooling': 'cov', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'emb_time_per_sample': 6.942558288574219e-05, 'inf_time_per_sample': 0.00039804840087890626, 'acc': 0.281, 'precision_k': 0.24801875}


### GeM-1

In [9]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'gem', 'faiss-lsh',
                         random_state=seed, k=k, ref_size=set_size_median, code_length=code_length, power=1)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 48708.00it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-1', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 1, 'emb_time_per_sample': 4.2551398277282715e-05, 'inf_time_per_sample': 0.0003985496997833252, 'acc': 0.1086, 'precision_k': 0.1044}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 53140.89it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-1', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 1, 'emb_time_per_sample': 3.8455986976623534e-05, 'inf_time_per_sample': 0.0003771803855895996, 'acc': 0.1042, 'precision_k': 0.1023375}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'gem-1', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 1, 'emb_time_per_sample': 3.865180015563965e-05, 'inf_time_per_sample': 0.0003745671033859253, 'acc': 0.1086, 'precision_k': 0.1044}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 55000.12it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-1', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 1, 'emb_time_per_sample': 3.635149002075195e-05, 'inf_time_per_sample': 0.00037083740234375, 'acc': 0.1042, 'precision_k': 0.1023375}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 55952.78it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-1', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'power': 1, 'emb_time_per_sample': 3.6765599250793455e-05, 'inf_time_per_sample': 0.00036921517848968505, 'acc': 0.1004, 'precision_k': 0.101225}


### GeM-2

In [10]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'gem', 'faiss-lsh',
                         random_state=seed, k=k, ref_size=set_size_median, code_length=code_length, power=2)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 26149.34it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-2', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 2, 'emb_time_per_sample': 5.6570601463317874e-05, 'inf_time_per_sample': 0.00037008697986602783, 'acc': 0.3214, 'precision_k': 0.28705}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 33002.08it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-2', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 2, 'emb_time_per_sample': 4.9127721786499024e-05, 'inf_time_per_sample': 0.00036777870655059817, 'acc': 0.3466, 'precision_k': 0.285675}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'gem-2', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 2, 'emb_time_per_sample': 5.3960394859313966e-05, 'inf_time_per_sample': 0.00036841652393341064, 'acc': 0.3214, 'precision_k': 0.28705}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 25742.54it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-2', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 2, 'emb_time_per_sample': 5.739588737487793e-05, 'inf_time_per_sample': 0.0003660981893539429, 'acc': 0.3466, 'precision_k': 0.285675}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 32724.23it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-2', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'power': 2, 'emb_time_per_sample': 5.104701519012451e-05, 'inf_time_per_sample': 0.0003694002866744995, 'acc': 0.3705, 'precision_k': 0.28901875}


### GeM-4

In [11]:
for seed in seeds:
    for k in ks:
        exp = Experiment(dataset, 'gem', 'faiss-lsh',
                         random_state=seed, k=k, ref_size=set_size_median, code_length=code_length, power=4)
        exp.test()
        report = exp.get_exp_report()
        print(report)
        reports.append(report)

loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 15978.04it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-4', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 4, 'emb_time_per_sample': 8.341200351715088e-05, 'inf_time_per_sample': 0.00037127938270568845, 'acc': 0.4458, 'precision_k': 0.393575}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 16204.54it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-4', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 4, 'emb_time_per_sample': 8.194692134857178e-05, 'inf_time_per_sample': 0.0003683065176010132, 'acc': 0.4711, 'precision_k': 0.3903}
loading dataset...
loading cached base embedding...
compute query embedding...


...

{'dataset': 'point_mnist', 'pooling': 'gem-4', 'ann': 'faiss-lsh', 'k': 4, 'code_length': 1024, 'power': 4, 'emb_time_per_sample': 7.612059116363525e-05, 'inf_time_per_sample': 0.0003658282041549683, 'acc': 0.4458, 'precision_k': 0.393575}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17514.51it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-4', 'ann': 'faiss-lsh', 'k': 8, 'code_length': 1024, 'power': 4, 'emb_time_per_sample': 7.680182456970215e-05, 'inf_time_per_sample': 0.0003727047920227051, 'acc': 0.4711, 'precision_k': 0.3903}
loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17511.32it/s]


{'dataset': 'point_mnist', 'pooling': 'gem-4', 'ann': 'faiss-lsh', 'k': 16, 'code_length': 1024, 'power': 4, 'emb_time_per_sample': 7.557501792907715e-05, 'inf_time_per_sample': 0.00036609373092651366, 'acc': 0.487, 'precision_k': 0.38306875}


In [12]:
import altair as alt

In [13]:
labels = {'fs': 'FSPool', 'swe': 'SLOSH', 'we': 'WE', 
          'cov': 'Cov', 'gem-1': 'GeM-1', 'gem-2': 'GeM-2', 'gem-4': 'GeM-4'}

In [14]:
data = pd.DataFrame(reports)
data['pooling'] = data['pooling'].apply(lambda x: labels[x])

In [15]:
points = alt.Chart(data).mark_point().encode(
    alt.X('mean(emb_time_per_sample):Q', title='Average Embedding Time'),
    alt.Y('mean(acc):Q', title='Accuracy'),
    color=alt.Color('pooling:N', legend=None),
).properties(
    width=240,
    height=240
)

In [16]:
text = points.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    size=15
).encode(
    text='pooling:N'
)

In [17]:
alt.layer(points + text).configure_axis(
    labelFontSize=12,
    titleFontSize=16
)

alt.LayerChart(...)

In [18]:
pd.options.display.float_format = "{:,.2f}".format

In [19]:
data.groupby(['pooling', 'k'])[['precision_k', 'acc']].mean()

precision_k  acc
pooling k                   
Cov     4          0.25 0.26
        8          0.25 0.28
        16         0.25 0.28
FSPool  4          0.75 0.80
        8          0.74 0.81
        16         0.72 0.81
GeM-1   4          0.10 0.11
        8          0.10 0.10
        16         0.10 0.10
GeM-2   4          0.29 0.32
        8          0.29 0.35
        16         0.29 0.37
GeM-4   4          0.39 0.45
        8          0.39 0.47
        16         0.38 0.49
SLOSH   4          0.89 0.92
        8          0.88 0.92
        16         0.86 0.91
WE      4          0.89 0.92
        8          0.88 0.92
        16         0.86 0.92